In [ ]:
# Importing Modules
import os
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn   
from torch.optim import Adam
from matplotlib import pyplot as plt
import pandas as pd
from matplotlib import pyplot as plt


In [ ]:
# Cudnn for internal optimization
torch.backends.cudnn.benchmark = True

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# Dataset Path
path = r'../input/digit-recognizer'
outputPath = r'../input/output/'

In [ ]:
# Reading the Data
train = pd.read_csv(os.path.join(path,'train.csv'))
test = pd.read_csv(os.path.join(path,'test.csv'))
sample_submission = pd.read_csv(os.path.join(path,'sample_submission.csv'))

## Lets understand the data

In [ ]:
# The label field is for label of the data
# The other fields are pixel values ranging from 0 to 255
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission.head()

In [ ]:
trainY = train.pop('label')
testY = pd.get_dummies(sample_submission.Label)
trainX = train.copy()
testX = test.copy()

In [ ]:
trainX

In [ ]:
trainY

In [ ]:
testY

In [ ]:
testX

In [ ]:
testY

## Dataset Class

In [ ]:
# Load the model and the dataset in the GPU(if any) , will help to speed up

In [ ]:
class datasetClass(Dataset):
    
    def __init__(self,):
        super(datasetClass,self).__init__()
        self.trainX = torch.tensor(trainX.values).type(torch.float32).div(255)
        self.trainX = self.trainX.sub_(0.1307).div_(0.3081).to(device)
        self.trainY = torch.tensor(trainY.values).type(torch.long).to(device)

    def __getitem__(self,index):
        return torch.tensor(self.trainX[index]),self.trainY[index]
    
    def __len__(self,):
        return len(self.trainY)

In [ ]:
datasetClassObj = datasetClass()

In [ ]:
iterObj= iter(datasetClassObj)

In [ ]:
x,y = next(iterObj)

In [ ]:
class datasetClassValid(Dataset):
    
    def __init__(self,):
        super(datasetClassValid,self).__init__()
        self.testX = torch.tensor(testX.values).type(torch.float32).div(255).sub_(0.1307).div_(0.3081).to(device)
        self.testY = torch.tensor(testY.values).type(torch.long).to(device)
        
    def __getitem__(self,index):
        return self.testX[index],self.testY[index]
    
    def __len__(self,):
        return len(self.testY)

In [ ]:
datasetClassValidObj = datasetClassValid()

In [ ]:
iterObj= iter(datasetClassValidObj)

In [ ]:
x,y = next(iterObj)

##  Lets build a model

In [ ]:
class modelClass(nn.Module):
    def __init__(self,inputDim,outputDim):
        super(modelClass,self).__init__()
        self.linearLayer1 = nn.Linear(inputDim,inputDim)
        self.batchnormLayer1  = nn.BatchNorm1d(inputDim)
        self.reluLayer1 = nn.ReLU()
        self.linearLayer2 = nn.Linear(inputDim,outputDim)
        self.softmaxLayer = nn.LogSoftmax()

    def forward(self,x):
        x = self.linearLayer1(x)
        x = self.batchnormLayer1(x)
        x = self.reluLayer1(x)
        x = self.linearLayer2(x)
        x = self.softmaxLayer(x)
        return x

In [ ]:
bs = 64
epochs = 50

In [ ]:
Model = modelClass(784,10)
Model.to(device)

In [ ]:
dataloaderTrain = DataLoader(datasetClassObj,batch_size=bs,shuffle=True)

In [ ]:
optimizer =  Adam(Model.parameters(),lr=0.01)
lossFunc = nn.NLLLoss()
loss  =  torch.tensor(0.1,requires_grad=True)

In [ ]:
losses = []
for epoch in range(epochs):
    # This step is very important
    Model.train()
    for x,y in dataloaderTrain:
        ydash = Model(x) 
        loss = lossFunc(ydash,y)
        loss.backward()
        with torch.no_grad():
            optimizer.step()
            optimizer.zero_grad()
    losses.append(loss)
    print(f'The epoch {epoch} the Loss is {loss}')


In [ ]:
# Loss Graph
cpuLoss = [float(loss.to('cpu')) for loss in losses] 
plt.plot(cpuLoss)

In [ ]:
dataloaderValidObj = DataLoader(datasetClassValidObj,batch_size=64,shuffle=False)

In [ ]:
plt.imshow(datasetClassValidObj.testX[0].to('cpu').view(28,28))

In [ ]:
submitframe = pd.DataFrame(columns=['Label'])

In [ ]:
for x,y in dataloaderValidObj:
    Model.eval()
    #plt.imshow(x.to('cpu').view(28,28))
    prediction = torch.argmax(Model(x),dim=1).to('cpu').numpy()
    submitframe = submitframe.append(pd.DataFrame(prediction,columns=['Label']))


In [ ]:
submitframe.head()

In [ ]:
submitframe.shape

In [ ]:
submitframe.to_csv('sample_submission_Result.csv')

## Next Steps 

1. The current model accuracy is 98%,to check the addition of convNet improves it further.
2. Addition of pytorch's transform function in the dataset creator.
3. Introduction of LeNet architechture to improve the prediction.
